# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напишите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))

if module_path not in sys.path:
    sys.path.append(module_path)
    
from IPython.display import display, HTML

os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1" 

import warnings
warnings.simplefilter('ignore')

# user's libs
from my_recommenders import MainRecommender
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items


data = pd.read_csv('../ex2/webinar_2/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id','product_id': 'item_id'}, inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

display(data_train.head(2))

item_features = pd.read_csv('../ex2/webinar_2/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

display(item_features.head(2))

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
display(result.head(2))

n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, 
                             group_col='item_id',
                             popular_col='quantity',
                             time_col='day',
                             price_col='sales_value',
                             top_popular_filter=5,
                             top_unpopular_filter=200,
                             time_unpopular_filter=365,
                             chip_item_filter=1,
                             exp_item_filter=600)

n_items_after = data_train['item_id'].nunique()
print('\nDecreased # items from {} to {}'.format(n_items_before, n_items_after))



my_recommender = MainRecommender(data_train)


similar_items_rec = my_recommender.get_similar_items_recommendation(355, N=5)
display(similar_items_rec)
display(item_features[item_features['item_id'].isin(similar_items_rec)])

similar_user_rec = my_recommender.get_similar_users_recommendation(443, N=5)
display(similar_user_rec)
display(item_features[item_features['item_id'].isin(similar_user_rec)])


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."



Decreased # items from 86865 to 51876


[1094262, 10122676, 9677378, 5651396, 5567194]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
36883,1094262,69,GROCERY,Private,SPICES & EXTRACTS,SPICES & SEASONINGS,1.75 OZ
52780,5567194,544,GROCERY,National,BAG SNACKS,TORTILLA/NACHO CHIPS,13 OZ
54619,5651396,103,GROCERY,National,SOFT DRINKS,SOFT DRINKS 12/18&15PK CAN CAR,12 OZ
69009,9677378,69,GROCERY,Private,REFRGRATD DOUGH PRODUCTS,REFRIGERATED SPECILATY ROLLS,17.5 OZ
71432,10122676,544,GROCERY,National,BAG SNACKS,POTATO CHIPS,11.5 OZ


[999999, 938118, 1044275, 12648296, 5592270]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
19417,938118,3722,DELI,National,DELI MEATS,MEAT:HAM BULK,
31324,1044275,69,PASTRY,Private,ROLLS,ROLLS: SANDWICH,
54481,5592270,69,GROCERY,Private,BAKED BREAD/BUNS/ROLLS,PREMIUM BREAD,24 OZ
78266,12648296,1722,GROCERY,National,FROZEN PIZZA,PIZZA/TRADITIONAL,21.6 OZ
